## LELEC2870 - Machine Learning Project
### Heart Failure Prediction in Smurf Society
---

# Part 3 - Integration of Image Data

---
---

## 1. Dataset Preparation

### Import necessary libraries

In [22]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split, KFold, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression as mutual_info
from sklearn.feature_selection import SequentialFeatureSelector

import random
import scipy.io

import warnings
warnings.filterwarnings('ignore')


from PIL import Image  # "PIL" stands for the "pillow" library.  
# Pillow is a dependency of pytorch. Hence, if you installed pytorch correctly, you should not have anything else to install.

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset



### Load the data

In [3]:
X_train = pd.read_csv('data/data_labeled/X_train.csv')
X_test = pd.read_csv('data/data_labeled/X_test.csv')

# y CSV files don't have headers, so we need to specify header=None and provide column names
y_train = pd.read_csv('data/data_labeled/y_train.csv', header=None, names=['heart_failure_risk'])
y_test = pd.read_csv('data/data_labeled/y_test.csv', header=None, names=['heart_failure_risk'])

# Extract image paths before dropping the column
img_paths_train = [os.path.join('data/data_labeled/Img_train', name) for name in X_train['img_filename'].values]
img_paths_test = [os.path.join('data/data_labeled/Img_test', name) for name in X_test['img_filename'].values]

# Explore the data
print(X_train.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                1000 non-null   int64  
 1   blood pressure     1000 non-null   float64
 2   calcium            1000 non-null   float64
 3   cholesterol        1000 non-null   float64
 4   hemoglobin         1000 non-null   float64
 5   height             1000 non-null   float64
 6   potassium          1000 non-null   float64
 7   profession         1000 non-null   object 
 8   sarsaparilla       1000 non-null   object 
 9   smurfberry liquor  1000 non-null   object 
 10  smurfin donuts     1000 non-null   object 
 11  vitamin D          1000 non-null   float64
 12  weight             1000 non-null   float64
 13  img_filename       1000 non-null   object 
dtypes: float64(8), int64(1), object(5)
memory usage: 109.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 en

In [4]:
# delete missing values in y_train
y_train = y_train.dropna()

# print length of y_train
print(len(y_train))

X_train = X_train.drop(columns=['img_filename'])
X_test = X_test.drop(columns=['img_filename'])

X_train = pd.get_dummies(X_train, columns=['profession'])
X_test = pd.get_dummies(X_test, columns=['profession'])

print(X_train.info())
print(X_test.info())

1000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   age                                       1000 non-null   int64  
 1   blood pressure                            1000 non-null   float64
 2   calcium                                   1000 non-null   float64
 3   cholesterol                               1000 non-null   float64
 4   hemoglobin                                1000 non-null   float64
 5   height                                    1000 non-null   float64
 6   potassium                                 1000 non-null   float64
 7   sarsaparilla                              1000 non-null   object 
 8   smurfberry liquor                         1000 non-null   object 
 9   smurfin donuts                            1000 non-null   object 
 10  vitamin D                       

In [5]:
# Strip whitespace from the three columns
X_train['sarsaparilla'] = X_train['sarsaparilla'].str.strip()
X_test['sarsaparilla'] = X_test['sarsaparilla'].str.strip()
X_train['smurfberry liquor'] = X_train['smurfberry liquor'].str.strip()
X_test['smurfberry liquor'] = X_test['smurfberry liquor'].str.strip()
X_train['smurfin donuts'] = X_train['smurfin donuts'].str.strip()
X_test['smurfin donuts'] = X_test['smurfin donuts'].str.strip()


X_train['sarsaparilla'] = X_train['sarsaparilla'].map({'Very high': 4, 'High': 3, 'Moderate': 2, 'Low': 1, 'Very low': 0})
X_test['x'] = X_test['sarsaparilla'].map({'Very high': 4, 'High': 3, 'Moderate': 2, 'Low': 1, 'Very low': 0})
X_train['smurfberry liquor'] = X_train['smurfberry liquor'].map({'Very high': 4, 'High': 3, 'Moderate': 2, 'Low': 1, 'Very low': 0})
X_test['smurfberry liquor'] = X_test['smurfberry liquor'].map({'Very high': 4, 'High': 3, 'Moderate': 2, 'Low': 1, 'Very low': 0})
X_train['smurfin donuts'] = X_train['smurfin donuts'].map({'Very high': 4, 'High': 3, 'Moderate': 2, 'Low': 1, 'Very low': 0})
X_test['smurfin donuts'] = X_test['smurfin donuts'].map({'Very high': 4, 'High': 3, 'Moderate': 2, 'Low': 1, 'Very low': 0})

print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   age                                       1000 non-null   int64  
 1   blood pressure                            1000 non-null   float64
 2   calcium                                   1000 non-null   float64
 3   cholesterol                               1000 non-null   float64
 4   hemoglobin                                1000 non-null   float64
 5   height                                    1000 non-null   float64
 6   potassium                                 1000 non-null   float64
 7   sarsaparilla                              1000 non-null   int64  
 8   smurfberry liquor                         1000 non-null   int64  
 9   smurfin donuts                            1000 non-null   int64  
 10  vitamin D                            

### Standardization

In [6]:
# Standardize only numerical features (not ordinal or one-hot encoded)
numerical_features = ['age', 'blood pressure', 'calcium', 'cholesterol', 'hemoglobin', 
                      'height', 'potassium', 'vitamin D', 'weight']

# Create a copy to avoid modifying original data
X_train_standardized = X_train.copy()
X_test_standardized = X_test.copy()

# Standardize only numerical features
scX = StandardScaler()
scX.fit(X_train[numerical_features])
X_train_standardized[numerical_features] = scX.transform(X_train[numerical_features])
X_test_standardized[numerical_features] = scX.transform(X_test[numerical_features])

# Update X_train and X_test
X_train = X_train_standardized
X_test = X_test_standardized

## 2. Optimisation

### CNN

In [7]:
# Compute the Root Mean Square Error
def compute_rmse(predict, target):

    len_predict = len(predict)
    len_target = len(target)
    
    if len_predict != len_target:
        raise ValueError("predict and target must have the same length")
    
    rmse = np.sqrt(np.mean((predict - target) ** 2))
    
    
    return rmse

In [8]:
# Cross-Validation Setup
rkf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
print(f"RepeatedKFold: {rkf.get_n_splits()} total folds (10 splits x 5 repeats)")

RepeatedKFold: 10 total folds (10 splits x 5 repeats)


In [9]:
# Custom Dataset for multimodal data (images + tabular)
# Optimized: Accepts pre-loaded images to avoid loading them multiple times
class HeartDataset(Dataset):
    def __init__(self, X_tabular, y, preloaded_images=None, image_paths=None, transform=None):
        # Ensure tabular data is a float32 NumPy array (no object dtype)
        if hasattr(X_tabular, 'to_numpy'):
            X_np = X_tabular.to_numpy(dtype=np.float32)
        else:
            X_np = np.asarray(X_tabular, dtype=np.float32)
        
        if hasattr(y, 'to_numpy'):
            y_np = y.to_numpy(dtype=np.float32).reshape(-1)
        else:
            y_np = np.asarray(y, dtype=np.float32).reshape(-1)

        self.X_tabular = torch.from_numpy(X_np)
        self.y = torch.from_numpy(y_np)
        
        # Use pre-loaded images if provided, otherwise load them
        if preloaded_images is not None:
            self.images = preloaded_images
        elif image_paths is not None:
            # Fallback: load images if preloaded_images not provided
            self.images = []
            for path in image_paths:
                img = Image.open(path).convert('L')
                if transform:
                    img = transform(img)
                self.images.append(img)
        else:
            raise ValueError("Either preloaded_images or image_paths must be provided")

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # Return pre-loaded image (no file I/O here!)
        return self.images[idx], self.X_tabular[idx], self.y[idx]


# Multimodal CNN: processes image through convolutions, then fuses with tabular data
class MultiModalCNN(nn.Module):
    def __init__(self, n_tabular_features, n_neurons=64):
        super(MultiModalCNN, self).__init__()
        
        # Image branch (input: 1 x 48 x 48)
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1)  # -> 48x48
        self.pool1 = nn.MaxPool2d(2)                                       # -> 24x24
        self.conv2 = nn.Conv2d(8, 8, kernel_size=3, stride=1, padding=1)   # -> 24x24
        self.pool2 = nn.MaxPool2d(2)                                       # -> 12x12
        
        self.flat_size = 8 * 12 * 12  # = 1152
        
        # Fusion layers
        self.fc1 = nn.Linear(self.flat_size + n_tabular_features, n_neurons)
        self.fc2 = nn.Linear(n_neurons, 1)
        self.relu = nn.ReLU()
        
    def forward(self, image, tabular):
        # Image path
        x = self.relu(self.conv1(image))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, self.flat_size)
        
        # Fuse with tabular data
        x = torch.cat((x, tabular), dim=1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Training functions for K-Fold Cross-Validation

# Set device: MPS (Apple Silicon) > CUDA (NVIDIA) > CPU
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA (NVIDIA GPU)")
else:
    device = torch.device('cpu')
    print("Using CPU")
print(f"Device: {device}\n")

def train_model(model, train_loader, criterion, optimizer, epochs, device):
    """Train the model for specified epochs."""
    model.train()
    model.to(device)
    
    n_batches = len(train_loader)
    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch_idx, (imgs, tabs, labels) in enumerate(train_loader):
            imgs, tabs, labels = imgs.to(device), tabs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(imgs, tabs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / n_batches
        print(f"    Epoch {epoch+1}/{epochs} - Avg Loss: {avg_loss:.6f}")

def evaluate_model(model, val_loader, device):
    """Evaluate model and return predictions and targets."""
    model.eval()
    model.to(device)
    
    preds, targets = [], []
    with torch.no_grad():
        for imgs, tabs, labels in val_loader:
            imgs, tabs, labels = imgs.to(device), tabs.to(device), labels.to(device)
            
            outputs = model(imgs, tabs).squeeze()
            preds.append(outputs.cpu().numpy())
            targets.append(labels.cpu().numpy())
    
    return np.concatenate(preds), np.concatenate(targets)

def preload_images(image_paths, transform):
    """Pre-load all images into memory once. Returns list of transformed tensors."""
    print(f"Pre-loading {len(image_paths)} images into memory...")
    images = []
    for i, path in enumerate(image_paths):
        img = Image.open(path).convert('L')  # Grayscale
        if transform:
            img = transform(img)
        images.append(img)
        if (i + 1) % 100 == 0:
            print(f"  Loaded {i + 1}/{len(image_paths)} images...")
    print(f"Finished loading all {len(image_paths)} images.\n")
    return images

def run_cross_validation(X_train, y_train, img_paths_train, cv_splitter, 
                         model_class, n_tabular_features, 
                         n_neurons=16, batch_size=32, lr=0.001, epochs=10):
    """Run cross-validation and return list of RMSEs for each fold."""
    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    
    # Pre-load ALL images ONCE before CV loop (saves massive time!)
    print("="*60)
    print("PRE-LOADING ALL IMAGES (done once before CV)")
    print("="*60)
    all_preloaded_images = preload_images(img_paths_train, transform)
    
    fold_rmses = []
    n_folds = cv_splitter.get_n_splits()
    
    print(f"Starting Cross-Validation:")
    print(f"  Total samples: {len(X_train)}")
    print(f"  Total folds: {n_folds}")
    print(f"  Hyperparameters: n_neurons={n_neurons}, batch_size={batch_size}, lr={lr}, epochs={epochs}\n")
    
    for fold_idx, (train_idx, val_idx) in enumerate(cv_splitter.split(X_train)):
        print(f"\n{'='*60}")
        print(f"Fold {fold_idx+1}/{n_folds}")
        print(f"  Train samples: {len(train_idx)}, Val samples: {len(val_idx)}")
        
        # Split data
        X_tr = X_train.iloc[train_idx]
        X_val = X_train.iloc[val_idx]
        y_tr = y_train.iloc[train_idx]
        y_val = y_train.iloc[val_idx]
        
        # Get pre-loaded images for this fold (just slice the pre-loaded list)
        img_tr = [all_preloaded_images[i] for i in train_idx]
        img_val = [all_preloaded_images[i] for i in val_idx]
        
        # Create datasets using pre-loaded images (no file I/O!)
        print(f"  Creating datasets...")
        train_dataset = HeartDataset(X_tr, y_tr, preloaded_images=img_tr)
        val_dataset = HeartDataset(X_val, y_val, preloaded_images=img_val)
        
        # Create data loaders (num_workers=0 to avoid multiprocessing issues in notebooks)
        num_workers = 0  # Set to 0 for Jupyter notebooks (avoids pickling errors)
        train_loader = DataLoader(
            train_dataset, 
            batch_size=batch_size, 
            shuffle=True,
            num_workers=num_workers
        )
        val_loader = DataLoader(
            val_dataset, 
            batch_size=batch_size,
            num_workers=num_workers
        )
        
        print(f"  Data loaders created")
        
        # Initialize model
        model = model_class(n_tabular_features=n_tabular_features, n_neurons=n_neurons)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        
        # Train
        print(f"  Training model...")
        train_model(model, train_loader, criterion, optimizer, epochs, device)
        
        # Evaluate
        print(f"  Evaluating on validation set...")
        preds, targets = evaluate_model(model, val_loader, device)
        rmse = compute_rmse(preds, targets)
        fold_rmses.append(rmse)
        
        print(f"  Fold {fold_idx+1} RMSE: {rmse:.4f}")
    
    return fold_rmses

# Run cross-validation
N_NEURONS = 16
BATCH_SIZE = 32
LR = 0.002
EPOCHS = 35

fold_rmses = run_cross_validation(
    X_train=X_train,
    y_train=y_train,
    img_paths_train=img_paths_train,
    cv_splitter=rkf,
    model_class=MultiModalCNN,
    n_tabular_features=X_train.shape[1],
    n_neurons=N_NEURONS,
    batch_size=BATCH_SIZE,
    lr=LR,
    epochs=EPOCHS
)

print(f"\nAverage RMSE: {np.mean(fold_rmses):.4f} (+/- {np.std(fold_rmses):.4f})")



Using MPS (Apple Silicon GPU)
Device: mps

PRE-LOADING ALL IMAGES (done once before CV)
Pre-loading 1000 images into memory...
  Loaded 100/1000 images...
  Loaded 200/1000 images...
  Loaded 300/1000 images...
  Loaded 400/1000 images...
  Loaded 500/1000 images...
  Loaded 600/1000 images...
  Loaded 700/1000 images...
  Loaded 800/1000 images...
  Loaded 900/1000 images...
  Loaded 1000/1000 images...
Finished loading all 1000 images.

Starting Cross-Validation:
  Total samples: 1000
  Total folds: 10
  Hyperparameters: n_neurons=16, batch_size=32, lr=0.002, epochs=35


Fold 1/10
  Train samples: 800, Val samples: 200
  Creating datasets...
  Data loaders created
  Training model...
    Epoch 1/35 - Avg Loss: 0.005674
    Epoch 2/35 - Avg Loss: 0.003064
    Epoch 3/35 - Avg Loss: 0.002336
    Epoch 4/35 - Avg Loss: 0.001953
    Epoch 5/35 - Avg Loss: 0.001300
    Epoch 6/35 - Avg Loss: 0.001041
    Epoch 7/35 - Avg Loss: 0.000760
    Epoch 8/35 - Avg Loss: 0.000654
    Epoch 9/35 - 

### CNN (trained) + Random Forest

In [25]:
# Extract features from a TRAINED CNN (conv layers only, before FC)
def extract_features_from_cnn(model, images, device):
    """Run images through trained conv layers, return flattened features."""
    model.eval()
    features = []
    with torch.no_grad():
        for img in images:
            img = img.unsqueeze(0).to(device)
            x = model.relu(model.conv1(img))
            x = model.pool1(x)
            x = model.relu(model.conv2(x))
            x = model.pool2(x)
            x = x.view(-1, model.flat_size)
            features.append(x.cpu().numpy())
    return np.concatenate(features, axis=0)


# Pre-load images once (reuse from above if already loaded)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
print("Loading images...")
all_images = [transform(Image.open(p).convert('L')) for p in img_paths_train]
print(f"Loaded {len(all_images)} images.\n")


def run_cv_cnn_rf(X_train, y_train, all_images, cv_splitter,
                  n_neurons=16, batch_size=32, lr=0.001, cnn_epochs=10,
                  n_estimators=100, max_depth=10, pca_components=20):
    """
    CV: Train CNN on each fold, extract features with trained conv layers, 
    reduce with PCA, then Random Forest.
    
    pca_components: number of PCA components to reduce CNN features to (default 20)
                    This balances CNN features with tabular features (~18)
    """
    fold_rmses = []
    n_folds = cv_splitter.get_n_splits()
    n_tabular = X_train.shape[1]
    
    print("="*60)
    print("CNN + PCA + Random Forest Cross-Validation")
    print("="*60)
    print(f"  Folds: {n_folds}")
    print(f"  CNN: epochs={cnn_epochs}, lr={lr}, n_neurons={n_neurons}")
    print(f"  PCA: 1152 CNN features → {pca_components} components")
    print(f"  RF: n_estimators={n_estimators}, max_depth={max_depth}")
    print(f"  Final features: {pca_components} (CNN/PCA) + {n_tabular} (tabular) = {pca_components + n_tabular}")
    print("="*60 + "\n")
    
    for fold_idx, (train_idx, val_idx) in enumerate(cv_splitter.split(X_train)):
        print(f"--- Fold {fold_idx+1}/{n_folds} ---")
        print(f"  Train: {len(train_idx)} samples, Val: {len(val_idx)} samples")
        
        # Split
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        img_tr = [all_images[i] for i in train_idx]
        img_val = [all_images[i] for i in val_idx]
        
        # Step 1: Train CNN
        print(f"  [Step 1] Training CNN ({cnn_epochs} epochs)...")
        train_ds = HeartDataset(X_tr, y_tr, preloaded_images=img_tr)
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        
        model = MultiModalCNN(n_tabular_features=X_train.shape[1], n_neurons=n_neurons).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        criterion = nn.MSELoss()
        
        model.train()
        for epoch in range(cnn_epochs):
            epoch_loss = 0.0
            for imgs, tabs, labels in train_loader:
                imgs, tabs, labels = imgs.to(device), tabs.to(device), labels.to(device)
                optimizer.zero_grad()
                loss = criterion(model(imgs, tabs).squeeze(), labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            avg_loss = epoch_loss / len(train_loader)
            print(f"    Epoch {epoch+1}/{cnn_epochs} - Loss: {avg_loss:.6f}")
        
        # Step 2: Extract features with TRAINED conv layers
        print(f"  [Step 2] Extracting CNN features...")
        cnn_tr = extract_features_from_cnn(model, img_tr, device)
        cnn_val = extract_features_from_cnn(model, img_val, device)
        print(f"    Raw CNN features: {cnn_tr.shape[1]}")
        
        # Step 3: Reduce CNN features with PCA
        print(f"  [Step 3] Reducing with PCA (1152 → {pca_components})...")
        pca = PCA(n_components=pca_components)
        cnn_tr_pca = pca.fit_transform(cnn_tr)
        cnn_val_pca = pca.transform(cnn_val)
        variance_explained = sum(pca.explained_variance_ratio_) * 100
        print(f"    Variance explained: {variance_explained:.1f}%")
        
        # Combine with tabular
        X_tr_np = X_tr.to_numpy(dtype=np.float32)
        X_val_np = X_val.to_numpy(dtype=np.float32)
        X_combined_tr = np.concatenate([cnn_tr_pca, X_tr_np], axis=1)
        X_combined_val = np.concatenate([cnn_val_pca, X_val_np], axis=1)
        print(f"    Final features: {X_combined_tr.shape[1]} ({pca_components} CNN + {n_tabular} tabular)")
        
        y_tr_np = y_tr.to_numpy(dtype=np.float32).reshape(-1)
        y_val_np = y_val.to_numpy(dtype=np.float32).reshape(-1)
        
        # Step 4: Train Random Forest
        print(f"  [Step 4] Training Random Forest ({n_estimators} trees)...")
        rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1)
        rf.fit(X_combined_tr, y_tr_np)
        
        preds = rf.predict(X_combined_val)
        rmse = compute_rmse(preds, y_val_np)
        fold_rmses.append(rmse)
        print(f"  [Result] Fold {fold_idx+1} RMSE: {rmse:.4f}\n")
    
    print("="*60)
    print(f"FINAL: Average RMSE = {np.mean(fold_rmses):.4f} (+/- {np.std(fold_rmses):.4f})")
    print("="*60)
    return fold_rmses


# Run it
fold_rmses_rf = run_cv_cnn_rf(
    X_train=X_train,
    y_train=y_train,
    all_images=all_images,
    cv_splitter=rkf,
    n_neurons=4, # keep it at 4 !
    batch_size=32,
    lr=0.002,
    cnn_epochs=20, # keep it at 15 !
    n_estimators=300,
    max_depth=10,
    pca_components=20  # Reduce 1152 CNN features to 20, balancing with 18 tabular features
)



Loading images...
Loaded 1000 images.

CNN + PCA + Random Forest Cross-Validation
  Folds: 10
  CNN: epochs=20, lr=0.002, n_neurons=4
  PCA: 1152 CNN features → 20 components
  RF: n_estimators=300, max_depth=10
  Final features: 20 (CNN/PCA) + 18 (tabular) = 38

--- Fold 1/10 ---
  Train: 800 samples, Val: 200 samples
  [Step 1] Training CNN (20 epochs)...
    Epoch 1/20 - Loss: 0.006930
    Epoch 2/20 - Loss: 0.003763
    Epoch 3/20 - Loss: 0.003371
    Epoch 4/20 - Loss: 0.003188
    Epoch 5/20 - Loss: 0.002930
    Epoch 6/20 - Loss: 0.002925
    Epoch 7/20 - Loss: 0.002904
    Epoch 8/20 - Loss: 0.002795
    Epoch 9/20 - Loss: 0.002467
    Epoch 10/20 - Loss: 0.002592
    Epoch 11/20 - Loss: 0.002271
    Epoch 12/20 - Loss: 0.002150
    Epoch 13/20 - Loss: 0.002069
    Epoch 14/20 - Loss: 0.002084
    Epoch 15/20 - Loss: 0.002034
    Epoch 16/20 - Loss: 0.001976
    Epoch 17/20 - Loss: 0.001906
    Epoch 18/20 - Loss: 0.001823
    Epoch 19/20 - Loss: 0.001905
    Epoch 20/20 - Loss

### Ensemble: CNN + CNN-PCA-RF (50-50 blend)


#### Why Use Different CNN Configurations for the Two Models?

##### The Problem with Shared Weights

Using the same CNN weights for both the neural network predictor and the Random Forest feature extractor leads to suboptimal performance. This happens for two key reasons:

1. **Overfitting to the Prediction Task**: A CNN trained with 16 neurons over 35 epochs becomes highly specialized for direct prediction. Its convolutional layers learn features that are optimized for the final fully-connected layers, not for general-purpose feature extraction.

2. **Lazy Feature Extraction**: When the CNN has more capacity (16 neurons), the convolutional layers can "rely" on the dense layers to do more of the work. This produces less informative image features.

##### The Solution: Separate CNN Training

For the CNN+PCA+RF pipeline, we use a smaller, less-trained CNN (4 neurons, 20 epochs):

| Configuration | CNN (Direct Prediction) | CNN (Feature Extraction for RF) |
|---------------|------------------------|--------------------------------|
| Neurons | 16 | 4 |
| Epochs | 35 | 20 |
| Goal | Minimize prediction error | Extract robust image features |

##### Benefits of This Approach

1. **Better Feature Quality**: With only 4 neurons, the convolutional layers must work harder to compress useful information, producing richer, more discriminative features.

2. **Reduced Overfitting to Noise**: Fewer training epochs prevent the CNN from memorizing noise in the training data, leading to more generalizable features.

3. **Lower Error Correlation**: Since the two models are trained differently, they tend to make different mistakes. This is crucial for ensembling — the 50-50 blend only improves performance when the models' errors are uncorrelated.

##### Impact on Ensemble Performance

When errors are uncorrelated, averaging predictions reduces variance:

$$\text{RMSE}_{ensemble} \approx \text{RMSE}_{individual} \times \sqrt{\frac{1 + \rho}{2}}$$

Where $\rho$ is the correlation between model errors. Lower $\rho$ → greater improvement from ensembling.

In [30]:
def run_cv_ensemble(X_train, y_train, all_images, cv_splitter,
                    n_neurons_cnn=16, batch_size=32, lr=0.001, cnn_epochs=35,
                    n_neurons_rf=4, rf_cnn_epochs=20, n_estimators=300, max_depth=10, pca_components=20,
                    weight_cnn=0.5):
    """
    Ensemble: Train both CNN and CNN+PCA+RF, blend predictions 50-50.
    
    weight_cnn: weight for CNN predictions (default 0.5 = equal blend)
    """
    fold_rmses_cnn = []
    fold_rmses_rf = []
    fold_rmses_ensemble = []
    n_folds = cv_splitter.get_n_splits()
    n_tabular = X_train.shape[1]
    
    print("="*60)
    print("ENSEMBLE: CNN + CNN-PCA-RF (50-50 blend)")
    print("="*60)
    print(f"  Folds: {n_folds}")
    print(f"  Model 1 (CNN): epochs={cnn_epochs}, n_neurons={n_neurons_cnn}, lr={lr}")
    print(f"  Model 2 (CNN+PCA+RF): cnn_epochs={rf_cnn_epochs}, n_neurons={n_neurons_rf}")
    print(f"    PCA: 1152 → {pca_components}, RF: {n_estimators} trees, max_depth={max_depth}")
    print(f"  Blend weights: CNN={weight_cnn:.0%}, RF={1-weight_cnn:.0%}")
    print("="*60 + "\n")
    
    for fold_idx, (train_idx, val_idx) in enumerate(cv_splitter.split(X_train)):
        print(f"{'='*60}")
        print(f"Fold {fold_idx+1}/{n_folds}")
        print(f"  Train: {len(train_idx)}, Val: {len(val_idx)}")
        print(f"{'='*60}")
        
        # Split data
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        img_tr = [all_images[i] for i in train_idx]
        img_val = [all_images[i] for i in val_idx]
        
        y_val_np = y_val.to_numpy(dtype=np.float32).reshape(-1)
        
        # =============================================
        # MODEL 1: Full CNN
        # =============================================
        print(f"\n  [Model 1: CNN]")
        train_ds = HeartDataset(X_tr, y_tr, preloaded_images=img_tr)
        val_ds = HeartDataset(X_val, y_val, preloaded_images=img_val)
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=batch_size)
        
        model_cnn = MultiModalCNN(n_tabular_features=n_tabular, n_neurons=n_neurons_cnn).to(device)
        optimizer = optim.Adam(model_cnn.parameters(), lr=lr)
        criterion = nn.MSELoss()
        
        print(f"    Training ({cnn_epochs} epochs)...")
        model_cnn.train()
        for epoch in range(cnn_epochs):
            epoch_loss = 0.0
            for imgs, tabs, labels in train_loader:
                imgs, tabs, labels = imgs.to(device), tabs.to(device), labels.to(device)
                optimizer.zero_grad()
                loss = criterion(model_cnn(imgs, tabs).squeeze(), labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            if (epoch + 1) % 10 == 0 or epoch == 0:
                print(f"      Epoch {epoch+1}/{cnn_epochs} - Loss: {epoch_loss/len(train_loader):.6f}")
        
        # Get CNN predictions
        model_cnn.eval()
        preds_cnn = []
        with torch.no_grad():
            for imgs, tabs, _ in val_loader:
                imgs, tabs = imgs.to(device), tabs.to(device)
                preds_cnn.append(model_cnn(imgs, tabs).squeeze().cpu().numpy())
        preds_cnn = np.concatenate(preds_cnn)
        rmse_cnn = compute_rmse(preds_cnn, y_val_np)
        fold_rmses_cnn.append(rmse_cnn)
        print(f"    CNN RMSE: {rmse_cnn:.4f}")
        
        # =============================================
        # MODEL 2: CNN + PCA + Random Forest
        # =============================================
        print(f"\n  [Model 2: CNN + PCA + RF]")
        
        # Train a separate CNN for feature extraction
        model_rf = MultiModalCNN(n_tabular_features=n_tabular, n_neurons=n_neurons_rf).to(device)
        optimizer = optim.Adam(model_rf.parameters(), lr=lr)
        
        print(f"    Training CNN for feature extraction ({rf_cnn_epochs} epochs)...")
        model_rf.train()
        for epoch in range(rf_cnn_epochs):
            epoch_loss = 0.0
            for imgs, tabs, labels in train_loader:
                imgs, tabs, labels = imgs.to(device), tabs.to(device), labels.to(device)
                optimizer.zero_grad()
                loss = criterion(model_rf(imgs, tabs).squeeze(), labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            if (epoch + 1) % 10 == 0 or epoch == 0:
                print(f"      Epoch {epoch+1}/{rf_cnn_epochs} - Loss: {epoch_loss/len(train_loader):.6f}")
        
        # Extract features
        print(f"    Extracting CNN features + PCA...")
        cnn_tr = extract_features_from_cnn(model_rf, img_tr, device)
        cnn_val = extract_features_from_cnn(model_rf, img_val, device)
        
        # PCA
        pca = PCA(n_components=pca_components)
        cnn_tr_pca = pca.fit_transform(cnn_tr)
        cnn_val_pca = pca.transform(cnn_val)
        
        # Combine with tabular
        X_tr_np = X_tr.to_numpy(dtype=np.float32)
        X_val_np = X_val.to_numpy(dtype=np.float32)
        X_combined_tr = np.concatenate([cnn_tr_pca, X_tr_np], axis=1)
        X_combined_val = np.concatenate([cnn_val_pca, X_val_np], axis=1)
        y_tr_np = y_tr.to_numpy(dtype=np.float32).reshape(-1)
        
        # Train RF
        print(f"    Training Random Forest ({n_estimators} trees)...")
        rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1)
        rf.fit(X_combined_tr, y_tr_np)
        
        preds_rf = rf.predict(X_combined_val)
        rmse_rf = compute_rmse(preds_rf, y_val_np)
        fold_rmses_rf.append(rmse_rf)
        print(f"    CNN+PCA+RF RMSE: {rmse_rf:.4f}")
        
        # =============================================
        # ENSEMBLE: Blend 50-50
        # =============================================
        preds_ensemble = weight_cnn * preds_cnn + (1 - weight_cnn) * preds_rf
        rmse_ensemble = compute_rmse(preds_ensemble, y_val_np)
        fold_rmses_ensemble.append(rmse_ensemble)
        
        print(f"\n  >>> ENSEMBLE RMSE: {rmse_ensemble:.4f} <<<")
        print(f"      (CNN: {rmse_cnn:.4f}, RF: {rmse_rf:.4f})\n")
    
    # Final results
    print("\n" + "="*60)
    print("FINAL RESULTS")
    print("="*60)
    print(f"  CNN only:        {np.mean(fold_rmses_cnn):.4f} (+/- {np.std(fold_rmses_cnn):.4f})")
    print(f"  CNN+PCA+RF only: {np.mean(fold_rmses_rf):.4f} (+/- {np.std(fold_rmses_rf):.4f})")
    print(f"  ENSEMBLE (50-50): {np.mean(fold_rmses_ensemble):.4f} (+/- {np.std(fold_rmses_ensemble):.4f})")
    print("="*60)
    
    return fold_rmses_cnn, fold_rmses_rf, fold_rmses_ensemble


# Run ensemble
rmses_cnn, rmses_rf, rmses_ensemble = run_cv_ensemble(
    X_train=X_train,
    y_train=y_train,
    all_images=all_images,
    cv_splitter=rkf,
    n_neurons_cnn=16,
    batch_size=32,
    lr=0.002,
    cnn_epochs=35,
    n_neurons_rf=4,
    rf_cnn_epochs=20,
    n_estimators=300,
    max_depth=10,
    pca_components=20,
    weight_cnn=0.5  # 50% CNN + 50% RF
)


ENSEMBLE: CNN + CNN-PCA-RF (50-50 blend)
  Folds: 10
  Model 1 (CNN): epochs=35, n_neurons=16, lr=0.002
  Model 2 (CNN+PCA+RF): cnn_epochs=20, n_neurons=4
    PCA: 1152 → 20, RF: 300 trees, max_depth=10
  Blend weights: CNN=50%, RF=50%

Fold 1/10
  Train: 800, Val: 200

  [Model 1: CNN]
    Training (35 epochs)...
      Epoch 1/35 - Loss: 0.008525
      Epoch 10/35 - Loss: 0.002463
      Epoch 20/35 - Loss: 0.001758
      Epoch 30/35 - Loss: 0.001360
    CNN RMSE: 0.0498

  [Model 2: CNN + PCA + RF]
    Training CNN for feature extraction (20 epochs)...
      Epoch 1/20 - Loss: 0.138054
      Epoch 10/20 - Loss: 0.010736
      Epoch 20/20 - Loss: 0.006908
    Extracting CNN features + PCA...
    Training Random Forest (300 trees)...
    CNN+PCA+RF RMSE: 0.0402

  >>> ENSEMBLE RMSE: 0.0365 <<<
      (CNN: 0.0498, RF: 0.0402)

Fold 2/10
  Train: 800, Val: 200

  [Model 1: CNN]
    Training (35 epochs)...
      Epoch 1/35 - Loss: 0.008117
      Epoch 10/35 - Loss: 0.002016
      Epoch 20/